In [12]:
import pandas as pd
import os

In [13]:
root_path = os.path.dirname(os.path.dirname(os.getcwd()))

lila_test_set_path = os.path.join(root_path, "data", "lila-equate", "StressTest", "test.csv")

scripts_path =  os.path.join(root_path, "data", "lila", "03_cleaned", "scripts")

eval_path =  os.path.join(root_path, "data", "lila", "03_cleaned")

os.makedirs(scripts_path, exist_ok=True)

lila_df = pd.read_csv(lila_test_set_path)

lila_df.head(1)

,sample_index,premise,hypothesis,golden_label,reference_label,completion,lila_label,lila_script
0,5999,"To be considered for “ movie of the year, ” a ...","To be considered for “ movie of the year, ” a ...",contradiction,contradiction,list_appearance_premise = 1/4\nlist_appearance...,contradiction,ListScoreS1 = 1/4 \nListScoreS2 = 6/4\nif List...


In [21]:
lila_df.shape[0]

1217

In [14]:
lila_df.columns

Index(['sample_index', 'premise', 'hypothesis', 'golden_label',
       'reference_label', 'completion', 'lila_label', 'lila_script'],
      dtype='object')

In [7]:
for index, script in zip(lila_df["sample_index"].values, lila_df["lila_script"].values):
    with open(os.path.join(scripts_path, f"sample_{index}.py"), 'x') as f:
        f.write(script)

In [15]:
from src.utils import run_script
from src.code_quality_utils import validate_code_quality

count = 0
for idx, row in lila_df.iterrows():
    print(idx)
    sample_index = getattr(row, "sample_index")
    script_path = os.path.join(scripts_path, f"sample_{sample_index}.py")
    label, error_message = run_script(script_path)
    lila_df.loc[idx, "inferred_label"] = label
    lila_df.loc[idx, "error_message"] = error_message

    code_quality_scores, code_quality_resolutions = validate_code_quality(script_path)
    for score_key, score in code_quality_scores.items():
        lila_df.loc[idx, score_key] = score
    # lila_df.loc[idx, "cc_resolutions"] = "; ".join(code_quality_resolutions)

lila_df.to_excel(os.path.join(eval_path, "lila_evaluated.xlsx"), index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [16]:
from sklearn.metrics import accuracy_score, f1_score

print(accuracy_score(lila_df["golden_label"], lila_df["inferred_label"]))
print(f1_score(lila_df["golden_label"], lila_df["inferred_label"], average="weighted"))

0.9967132292522597
0.9967145754356701


In [17]:
lila_df[lila_df["golden_label"] != lila_df["inferred_label"]]

,sample_index,premise,hypothesis,golden_label,reference_label,completion,lila_label,lila_script,inferred_label,error_message,readability,document_size,redundancy_check,function_size
61,3046,Sheila works more than 5 hours per day on Mond...,"Sheila works 8 hours per day on Monday, Wednes...",neutral,neutral,work_hours_mon_wed_fri_premise = 5\nwork_hours...,neutral,sheila_works_s1 = 5\nsheila_works_s2 = 8\n \ni...,contradiction,,0.0,44.0,81.82,100.0
73,3732,Kiran travels from A to B by car and returns f...,Kiran travels from A to B by car and returns f...,entailment,entailment,travel_time_premise = 7\ntravel_time_hypothesi...,Entailment,Kiran_travels_s1=7\nKiran_travels_s2=8\n\nif K...,contradiction,,0.0,52.0,69.23,100.0
78,1955,How much loss would Indu has suffered had she ...,How much loss would Indu has suffered had she ...,contradiction,contradiction,years_premise = 2\nyears_hypothesis = 2\ninter...,contradiction,indu_loss_year_s1 = 2\nindu_loss_year_s2 = 2-1...,entailment,,0.0,44.0,81.82,100.0
951,7212,"Roslin played the game, getting at least one s...","Roslin played the game, getting at least one s...",entailment,neutral,lowest_score_premise = 1\nlowest_score_hypothe...,Entailment,score_S1 = 1\nscore_S2 = 8\nif score_S1 is Non...,contradiction,,0.0,32.0,100.00,100.0


In [19]:
bpm_results_df = pd.read_excel(os.path.join(root_path, "results_inference", "codellama-instruct", "13B", "6001211_finetuned", "evaluation_new", "qnli_classification_results.xlsx"))

bpm_stress_test_df = bpm_results_df[bpm_results_df["source"]=="stresstest"]

bpm_stress_test_df.shape[0]

1217

In [24]:
print(round(accuracy_score(bpm_stress_test_df["golden_label"], bpm_stress_test_df["inferred_label"]), 3))
print(round(f1_score(bpm_stress_test_df["golden_label"], bpm_stress_test_df["inferred_label"], average="weighted"), 3))

0.969
0.97


In [25]:
bpm_results_df = pd.read_excel(os.path.join(root_path, "results_inference", "codellama-instruct", "7B", "5892669_finetuned", "evaluation_new", "qnli_classification_results.xlsx"))

bpm_stress_test_df = bpm_results_df[bpm_results_df["source"]=="stresstest"]

bpm_stress_test_df.shape[0]

1217

In [26]:
print(round(accuracy_score(bpm_stress_test_df["golden_label"], bpm_stress_test_df["inferred_label"]), 3))
print(round(f1_score(bpm_stress_test_df["golden_label"], bpm_stress_test_df["inferred_label"], average="weighted"), 3))

0.95
0.951
